# Membuat Layanan Inferensi Batch

Bayangkan sebuah klinik kesehatan melakukan pengukuran pasien sepanjang hari, menyimpan detail untuk setiap pasien dalam file terpisah. Kemudian dalam semalam, model prediksi diabetes dapat digunakan untuk memproses semua data pasien hari itu sebagai batch, menghasilkan prediksi yang akan menunggu keesokan paginya sehingga klinik dapat menindaklanjuti pasien yang diprediksi berisiko diabetes. Dengan Azure Machine Learning, Anda dapat melakukannya dengan membuat *alur inferensi batch*; dan itulah yang akan Anda terapkan dalam latihan ini.

## Menghubungkan ke ruang kerja Anda

Untuk memulai, hubungkan ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Melatih dan mendaftarkan model

Sekarang mari kita latih dan daftarkan model untuk disebarkan dalam alur inferensi batch.

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name='mslearn-train-diabetes')
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

## Membuat dan mengunggah data batch

Karena kami sebenarnya tidak memiliki klinik dengan staf penuh dengan pasien yang akan mendapatkan data baru untuk latihan ini, Anda akan membuat sampel acak dari file CSV diabetes kami, mengunggah data tersebut ke datastore di ruang kerja Azure Machine Learning, dan mendaftarkan himpunan data untuk itu.

In [ ]:
from azureml.core import Datastore, Dataset
import pandas as pd
import os

# Set default data store
ws.set_default_datastore('workspaceblobstore')
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

# Load the diabetes data
diabetes = pd.read_csv('data/diabetes2.csv')
# Get a 100-item sample of the feature columns (not the diabetic label)
sample = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].sample(n=100).values

# Create a folder
batch_folder = './batch-data'
os.makedirs(batch_folder, exist_ok=True)
print("Folder created!")

# Save each sample as a separate file
print("Saving files...")
for i in range(100):
    fname = str(i+1) + '.csv'
    sample[i].tofile(os.path.join(batch_folder, fname), sep=",")
print("files saved!")

# Upload the files to the default datastore
print("Uploading files to datastore...")
default_ds = ws.get_default_datastore()
default_ds.upload(src_dir="batch-data", target_path="batch-data", overwrite=True, show_progress=True)

# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(default_ds, 'batch-data/'), validate=False)
try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='batch-data',
                                             description='batch data',
                                             create_new_version=True)
except Exception as ex:
    print(ex)

print("Done!")

## Memuat komputasi

Kita memerlukan konteks komputasi untuk alur, jadi kita akan menggunakan kode berikut untuk menentukan kluster komputasi Azure Machine Learning (itu akan dibuat jika belum ada).

> **Penting**: Ubah *your-compute-cluster* menjadi nama kluster komputasi Anda dalam kode di bawah ini sebelum menjalankannya! Nama kluster harus berupa nama unik global dengan panjang antara 2 hingga 16 karakter. Karakter yang valid adalah huruf, angka, dan - karakter.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        inference_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        inference_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

> **Catatan**: Instans dan kluster komputasi didasarkan pada gambar mesin virtual Azure standar. Untuk latihan ini, gambar *Standard_DS11_v2* direkomendasikan untuk mencapai keseimbangan optimal antara biaya dan performa. Jika langganan Anda memiliki kuota yang tidak menyertakan gambar ini, pilih gambar alternatif; tetapi perlu diingat bahwa gambar yang lebih besar mungkin memerlukan biaya lebih tinggi dan gambar yang lebih kecil mungkin tidak cukup untuk menyelesaikan tugas. Atau, minta administrator Azure Anda untuk memperbesar kuota Anda.

## Membuat alur untuk inferensi batch

Sekarang kita siap untuk mendefinisikan alur yang akan kita gunakan untuk inferensi batch. Alur kita akan membutuhkan kode Python untuk melakukan inferensi batch, jadi mari buat folder tempat kita dapat menyimpan semua file yang digunakan oleh alur:

In [ ]:
import os
# Create a folder for the experiment files
experiment_folder = 'batch_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

Sekarang kita akan membuat skrip Python untuk melakukan pekerjaan yang sebenarnya, dan menyimpannya di folder alur:

In [ ]:
%%writefile $experiment_folder/batch_diabetes.py
import os
import numpy as np
from azureml.core import Model
import joblib


def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)


def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read the comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for prediction (model expects multiple items)
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

Alur akan membutuhkan lingkungan untuk menjalankannya, jadi kami akan membuat spesifikasi Conda yang menyertakan paket yang digunakan kode.

In [ ]:
%%writefile $experiment_folder/batch_environment.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- pip
- pip:
  - azureml-defaults

Selanjutnya kita akan mendefinisikan konteks run yang menyertakan lingkungan Conda.

In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# Create an Environment for the experiment
batch_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/batch_environment.yml")
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

Anda akan menggunakan alur untuk menjalankan skrip prediksi batch, menghasilkan prediksi dari data input, dan menyimpan hasilnya sebagai file teks di folder output. Untuk melakukannya, Anda dapat menggunakan **ParallelRunStep**, yang memungkinkan data batch diproses secara paralel dan hasilnya dikumpulkan dalam satu file keluaran bernama *parallel_run_step.txt*.

In [ ]:
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig

output_dir = OutputFileDatasetConfig(name='inferences')

parallel_run_config = ParallelRunConfig(
    source_directory=experiment_folder,
    entry_script="batch_diabetes.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=inference_cluster,
    node_count=2)

parallelrun_step = ParallelRunStep(
    name='batch-score-diabetes',
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('diabetes_batch')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

Sekarang saatnya untuk memasukkan langkah ke dalam alur, dan menjalankannya.

> **Catatan**: Ini mungkin memakan waktu lama!

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
pipeline_run = Experiment(ws, 'mslearn-diabetes-batch').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

Saat alur selesai berjalan, prediksi yang dihasilkan akan disimpan dalam output eksperimen yang terkait dengan langkah pertama (dan satu-satunya) dalam alur. Anda dapat mengambilnya sebagai berikut:

In [ ]:
import pandas as pd
import shutil

# Remove the local results folder if left over from a previous run
shutil.rmtree('diabetes-results', ignore_errors=True)

# Get the run for the first step and download its output
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='diabetes-results')

# Traverse the folder hierarchy and find the results file
for root, dirs, files in os.walk('diabetes-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

## Menerbitkan Alur dan Menggunakan Antarmuka REST-nya

Sekarang setelah Anda memiliki alur yang berfungsi untuk inferensi batch, Anda dapat memublikasikannya dan menggunakan titik akhir REST untuk menjalankannya dari aplikasi.

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name='diabetes-batch-pipeline', description='Batch scoring of diabetes data', version='1.0')

published_pipeline

Perhatikan bahwa alur yang diterbitkan memiliki titik akhir, yang dapat Anda lihat di portal Azure. Anda juga dapat menemukannya sebagai properti dari objek alur yang diterbitkan:

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

Untuk menggunakan titik akhir, aplikasi klien perlu melakukan panggilan REST melalui HTTP. Permintaan ini harus diautentikasi, sehingga header otorisasi diperlukan. Untuk mengujinya, kami akan menggunakan header otorisasi dari koneksi Anda saat ini ke ruang kerja Azure Anda, yang bisa Anda dapatkan menggunakan kode berikut:

> **Catatan**: Aplikasi nyata memerlukan prinsip layanan yang akan diautentikasi.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print('Authentication header ready.')

Sekarang kita siap untuk memanggil antarmuka REST. Alur berjalan secara asinkron, jadi kita akan mendapatkan pengidentifikasi kembali, yang dapat kita gunakan untuk melacak eksperimen alur saat dijalankan:

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "mslearn-diabetes-batch"})
run_id = response.json()["Id"]
run_id

Karena kita memiliki ID proses, kita dapat menggunakan widget **RunDetails** untuk melihat eksperimen saat dijalankan:

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments['mslearn-diabetes-batch'], run_id)

# Block until the run completes
published_pipeline_run.wait_for_completion(show_output=True)

Tunggu hingga proses alur selesai, lalu jalankan sel berikut untuk melihat hasilnya.

Seperti sebelumnya, hasilnya ada di output dari langkah alur pertama:

In [ ]:
import pandas as pd
import shutil

# Remove the local results folder if left over from a previous run
shutil.rmtree('diabetes-results', ignore_errors=True)

# Get the run for the first step and download its output
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='diabetes-results')

# Traverse the folder hierarchy and find the results file
for root, dirs, files in os.walk('diabetes-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

Sekarang Anda memiliki alur yang dapat digunakan untuk memproses data pasien harian secara batch.

**Informasi Selengkapnya**: Untuk detail selengkapnya tentang penggunaan alur untuk inferensi batch, lihat [Cara Menjalankan Prediksi Batch](https://docs.microsoft.com/azure/machine-learning/how-to-run-batch-predictions) dalam dokumentasi Azure Machine Learning.